In [27]:
import pandas as pd
import geopandas as gpd
from shapely import wkb

from biz_opps.neo4j.helpers import get_neo4j_driver
from biz_opps.utils.postgres import get_sqlalchemy_engine
from biz_opps.utils.file import get_root_dir

driver = get_neo4j_driver()


Connected to Neo4j database successfully.


In [37]:
# gEt table names from postgres
# Connect to postgres
engine = get_sqlalchemy_engine()

# Query table names
query = "SELECT * from nourish_geoenrichment_variables LIMIT 10"

# Execute query and get table names
bgs_df = pd.read_sql(query, engine)

# print the crs of the geometry column
display(bgs_df)


,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
0,MP12206a_I,2024 Index: Drank Monster Energy Drink/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12206a_I,GroceryAlcoholicBeverages_MP12206a_I,2024 Drank Monster Energy Drink Last 6 Mo: Index,2024,count
1,MP12207a_B,2024 Drank Red Bull Energy Drink/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12207a_B,GroceryAlcoholicBeverages_MP12207a_B,2024 Drank Red Bull Energy Drink Last 6 Mo,2024,count
2,MP12207a_I,2024 Index: Drank Red Bull Energy Drink/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12207a_I,GroceryAlcoholicBeverages_MP12207a_I,2024 Drank Red Bull Energy Drink Last 6 Mo: Index,2024,count
3,MP12208a_B,2024 Drank Rockstar Energy Drink/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12208a_B,GroceryAlcoholicBeverages_MP12208a_B,2024 Drank Rockstar Energy Drink Last 6 Mo,2024,count
4,MP12208a_I,2024 Index: Drank Rockstar Energy Drink/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12208a_I,GroceryAlcoholicBeverages_MP12208a_I,2024 Drank Rockstar Energy Drink Last 6 Mo: Index,2024,count
5,MP12184a_B,2024 Drank Sports Drink or Thirst Quencher/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12184a_B,GroceryAlcoholicBeverages_MP12184a_B,2024 Drank Sports Drink or Thirst Quencher Las...,2024,count
6,MP12184a_I,2024 Index: Drank Sports Drink or Thirst Quenc...,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12184a_I,GroceryAlcoholicBeverages_MP12184a_I,2024 Drank Sports Drink or Thirst Quencher Las...,2024,count
7,MP12234h_B,2024 HH Used Orange Juice/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12234h_B,GroceryAlcoholicBeverages_MP12234h_B,2024 HH Used Orange Juice Last 6 Mo,2024,count
8,MP12234h_I,2024 Index: HH Used Orange Juice/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12234h_I,GroceryAlcoholicBeverages_MP12234h_I,2024 HH Used Orange Juice Last 6 Mo: Index,2024,count
9,MP12236h_B,2024 HH Used Oth Fruit Juice/Drink/6 Mo,GroceryAlcoholicBeverages,GroceryAlcoholicBeverages.MP12236h_B,GroceryAlcoholicBeverages_MP12236h_B,2024 HH Used Other Fruit Juice/Drink Last 6 Mo,2024,count


In [34]:
# Load bgs
bgs_df = pd.read_csv(f"{get_root_dir()}/data/bgs_sd_imp.csv")

# Convert wkb_geometry to shapely geometry and create GeoDataFrame
bgs_gdf = gpd.GeoDataFrame(
    bgs_df, 
    geometry=gpd.GeoSeries.from_wkb(bgs_df['wkb_geometry']), 
    crs=4326
)

# Convert geometry to WKT format
bgs_gdf['wkt'] = bgs_gdf.geometry.to_wkt()

# Query BlockGroup nodes from Neo
with driver.session() as session:
    query = """
    MATCH (bg:BlockGroup)
    RETURN bg.ct_block_group as ct_block_group, bg.wkt as wkt
    """
    result = session.run(query)
    neo4j_bgs = pd.DataFrame([dict(record) for record in result])

# Create ct_block_group in bgs_df by combining tract and block group, removing leading zeros
bgs_gdf['ct_block_group'] = bgs_gdf['tractce'].astype(str).str.lstrip('0') + bgs_gdf['blkgrpce'].astype(str).str.lstrip('0')

# Merge the dataframes to compare
comparison_df = pd.merge(
    bgs_gdf[['ct_block_group', 'wkt']], 
    neo4j_bgs,
    on='ct_block_group',
    suffixes=('_csv', '_neo4j')
)

# Check for matches
total_bgs = len(bgs_gdf)
matched_bgs = len(comparison_df)
wkt_matches = (comparison_df['wkt_csv'] == comparison_df['wkt_neo4j']).sum()

print(f"Total BlockGroups in CSV: {total_bgs}")
print(f"Matched BlockGroups: {matched_bgs}")
print(f"BlockGroups with matching WKT: {wkt_matches}")

# Show unmatched BlockGroups from CSV
unmatched_bgs = bgs_gdf[~bgs_gdf['ct_block_group'].isin(neo4j_bgs['ct_block_group'])]
if len(unmatched_bgs) > 0:
    print("\nUnmatched BlockGroups from CSV:")
    print(unmatched_bgs['ct_block_group'].tolist())


TypeError: cannot convert the series to <class 'int'>